In [26]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline 
from matplotlib.ticker import MultipleLocator

#from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D
from pylab import figure
#import seaborn as sns
#sns.set(style='white', context='poster', rc={'figure.figsize':(7,5)})

import pandas as pd
import csv
import numpy as np
import os 
import re 

import umap
from keras import constraints
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils import class_weight

from keras.models import load_model
from keras.utils.vis_utils import plot_model
from keras import initializers
from keras import regularizers
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import * # Keras is the most friendly Neural Network library, this Kernel use a lot of layers classes
from keras.models import Model
from keras import backend as K # The backend give us access to tensorflow operations and allow us to create the Attention class
from keras import optimizers # Allow us to access the Adam class to modify some parameters
from keras.callbacks import * # This object helps the model to train in a smarter way, avoiding overfitting

In [2]:
train = pd.read_csv("data/3Classes_train.csv", sep="\t")
valid = pd.read_csv("data/3Classes_valid.csv", sep="\t")

y_train = train["Global classifier2"]
y_valid = valid["Global classifier2"]

x_train = train["Sequence"].copy()
x_valid = valid["Sequence"].copy()

In [3]:
valid.head()

,Lead Protein ID,Global classifier2,Sequence,Sequence Lengh
0,Q96MN5,Mostly Organellar,MDKFVIRTPRIQNSPQKKDSGGKVYKQATIESLKRVVVVEDIKRWK...,208
1,P42575,Mostly Cytosolic,MAAPSAGSWSTFQHKELMAADRGRRILGVCGMHPHHQETLKKNRVV...,452
2,Q9NZP8,Mostly Organellar,MPGPRVWGKYLWRSPHSKGCPGAMWWLLLWGVLQACPTRGSVLLAQ...,487
3,Q96K80,Mostly Nuclear,MPDRDSYANGTGSSGGGPGGGGSEEASGAGVGSGGASSDAICRDFL...,434
4,Q86TA1,Mostly Cytosolic,MSIALKQVFNKDKTFRPKRKFEPGTQRFELHKRAQASLNSGVDLKA...,216


In [4]:
print(len(x_train), 'train sequences')
print(len(x_valid), 'test sequences')

4015 train sequences
709 test sequences


In [7]:
maxlen = len(max(x_train, key=len))

In [8]:
# all amino acids (in blosum order)
aa = "*ARNDCQEGHILKMFPSTWYVU"
tot_aa = len(aa)

# define a mapping of aa to integers
aa_to_int = dict((c, i) for i, c in enumerate(aa))
int_to_aa = dict((i, c) for i, c in enumerate(aa))

## One hot encoding of Labels

In [9]:
cat = ['Mostly Cytosolic', 'Mostly Nuclear', 'Mostly Organellar']

tot_cat = len(cat)
cat_to_int = {}
int_to_cat = {}
for i in range(tot_cat):
    cat_to_int[cat[i]] = i
    int_to_cat[i] = cat[i]   

In [10]:
def cat_to_indices(Y):
    Y_indices = np.zeros([Y.shape[0],], dtype=int)
    for i in range(len(Y)):
        Y_indices[i] = cat_to_int[Y[i]]
    return Y_indices

# one hot encode
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

In [11]:
y_train_indices = cat_to_indices(y_train)
y_valid_indices = cat_to_indices(y_valid)

y_train_OH = convert_to_one_hot(y_train_indices, C = tot_cat)
y_valid_OH = convert_to_one_hot(y_valid_indices, C = tot_cat)

## X_train processing: 
### From Sequences to list of indices

In [12]:
# Function to convert sequences to array of indices. I will that that one for embedding 

def seq_to_indices(X, aa_to_int, max_len):
    
    m = X.shape[0]                                   # number of training examples
    
    ### START CODE HERE ###
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = []
    
    for i in range(m):                               # loop over training examples
        seq_aa = X[i]
        seq_ind = []
        
        # Loop over the words of sentence_words
        for w in range(len(seq_aa)):
            seq_ind.append(aa_to_int[seq_aa[w]])
            
        X_indices.append(seq_ind)
            
    ### END CODE HERE ###
    
    return X_indices

In [14]:
x_train_indices = seq_to_indices(x_train, aa_to_int, maxlen)
x_valid_indices = seq_to_indices(x_valid, aa_to_int, maxlen)

In [15]:
# Pad sequences
# By default, the padding is added before the sequence
x_train_pad = sequence.pad_sequences(x_train_indices, maxlen=maxlen, value=aa_to_int["*"])
x_valid_pad = sequence.pad_sequences(x_valid_indices, maxlen=maxlen, value=aa_to_int["*"])
print('x_train shape:', x_train_pad.shape)
print('x_valid shape:', x_valid_pad.shape)

x_train shape: (4015, 1500)
x_valid shape: (709, 1500)


In [16]:
# One hot to label 
#Y is list of OH vector
def OH_to_label_indices(Y):
    labels = []
    for a in Y:
        indices = np.argmax(a)
        labels.append(indices)
    return labels

In [17]:
def indices_to_cat(Y):
    Y_cat = []
    for i in range(len(Y)):
        Y_cat.append(int_to_cat[Y[i]])
    return Y_cat

# First Model

Attention from this source: https://mlwhiz.com/blog/2019/03/09/deeplearning_architectures_text_classification/

In [23]:
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    

class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]


def model_lstm_atten(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    #x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)
    x = AttentionWithContext()(x)
    x = Dense(64, activation="relu")(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [24]:
model = load_model('models_attention/1/weights_embedding22.hdf5')
embedding = model.get_layer('embedding_layer').get_weights()
embedding = embedding[0]
embedding.shape

(22, 80)

In [27]:
max_features = tot_aa
embed_size = 80
units = 128

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)
d_class_weights = dict(enumerate(class_weights))

model = model_lstm_atten(embedding)

ckpt = ModelCheckpoint(filepath='models_tf_attention/weights_model1.hdf5', 
                           verbose=1, save_best_only=True)
# Train, train, train
history = model.fit(x_train_pad, y_train_OH, 
                    batch_size=64, epochs=40,
                    class_weight = class_weights,
          validation_data=[x_valid_pad, y_valid_OH], callbacks=[ckpt])

TypeError: add_weight() got multiple values for argument 'name'

In [ ]:
max_features = tot_aa
embed_size = 40
units = 128

_input = Input(shape=[max_len], dtype='int32')

# get the embedding layer
embedded = Embedding(
        input_dim=max_features,
        output_dim=embed_size,
        input_length=max_len,
        name = 'embedding_layer',
        trainable=True,
        mask_zero=False
    )(_input)

lstm = Bidirectional(LSTM(units, return_sequences=True, 
                                 dropout=0.2, recurrent_dropout=0.2))(embedded)

# compute importance for each step
attention = AttentionWithContext()(lstm)

dense = Dense(64, activation="relu")(attention)

probabilities = Dense(3, activation='softmax')(dense)

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Valid'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Valid'], loc='upper left')
plt.show()

In [ ]:
model = load_model('models_attention/1/weights_embedding22.hdf5')

In [ ]:
plot_model(model, to_file= 'models_attention/1/model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
model.evaluate(x_valid_pad, y_valid_OH)

In [ ]:
y_pred = model.predict(x_valid_pad)
y_pred_indices = OH_to_label_indices(y_pred)
y_pred_labels = indices_to_cat(y_pred_indices)

In [ ]:
def confusion_plot(y_valid, y_pred_labels, cat):
    conf_mx = confusion_matrix(y_valid, y_pred_labels, labels=cat)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(conf_mx, cmap=plt.cm.gray)
    fig.colorbar(cax)
    ax.xaxis.set_major_locator(MultipleLocator(1))
    ax.yaxis.set_major_locator(MultipleLocator(1))
    ax.set_xticklabels([''] + cat,  rotation='vertical')
    ax.set_yticklabels([''] + cat)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()
    
confusion_plot(y_valid, y_pred_labels, cat)

In [ ]:
conf_mx

In [ ]:
def confusion_plot_norm(y_valid, y_pred_labels, cat, axis=1):
    conf_mx = confusion_matrix(y_valid, y_pred_labels, labels=cat)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(conf_mx / (conf_mx.max(axis=axis)+1), cmap=plt.cm.gray)
    fig.colorbar(cax)
    ax.xaxis.set_major_locator(MultipleLocator(1))
    ax.yaxis.set_major_locator(MultipleLocator(1))
    ax.set_xticklabels([''] + cat,  rotation='vertical')
    ax.set_yticklabels([''] + cat)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()
    return(conf_mx)
conf = confusion_plot_norm(y_valid, y_pred_labels, cat, axis=1)

In [ ]:
print(classification_report(y_valid, y_pred_labels))

## Plot of the Embedding Layer

Good article about different dimension reduction techniques:
https://www.analyticsvidhya.com/blog/2018/08/dimensionality-reduction-techniques-python/

In [ ]:
embedding = model.get_layer('embedding_layer').get_weights()
embedding = embedding[0]
embedding.shape

### PCA plot

In [ ]:
# PCA plots of Embedding 
pca = PCA(n_components=2)
embedding_pca = pca.fit_transform(embedding)
print("original shape:   ", embedding.shape)
print("transformed shape:", embedding_pca.shape)

In [ ]:
def plot_embedding(embedding, aa):
    fig, ax = plt.subplots()
    plt.scatter(embedding[:, 0], embedding[:, 1], edgecolor='none')
    aaL = list(aa)
    for i in range(len(aaL)):
        ax.annotate(aaL[i], (embedding[:, 0][i], embedding[:, 1][i]))
    
    plt.show()

In [ ]:
plot_embedding(embedding_pca, aa)

### t-SNE plot

t-distributed stochastic neighbor embedding

Doc for tSNE: https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html

In [ ]:
# tSNE plots of Embedding 
tSNE = TSNE(n_components=2, perplexity=4)
embedding_tSNE = tSNE.fit_transform(embedding)

print("original shape:   ", embedding.shape)
print("transformed shape:", embedding_tSNE.shape)

# NB: perplexity plays a huge role there, 
#since our number of feature is very small I should use a very small value (between 2 and 7).

In [ ]:
plot_embedding(embedding_tSNE, aa)

 We can distinguish 3 main group:
 - Negatively charged
 - Hydrophobic 
 - Positively charged

It would be nice to color them... 
Look how to do it.

### Umap plot

Umap documentation: https://umap-learn.readthedocs.io/en/latest/parameters.html

n_neighbors is similar to perplexity for tSNE, important to optimize.

In [ ]:
# umap plots of Embedding 
Umap = umap.UMAP(n_components=2, n_neighbors= 5)
embedding_Umap = Umap.fit_transform(embedding)

print("original shape:   ", embedding.shape)
print("transformed shape:", embedding_Umap.shape)

In [ ]:
plot_embedding(embedding_Umap, aa)

In [ ]:
Umap3 = umap.UMAP(n_components=3, n_neighbors= 6)
embedding_Umap3 = Umap3.fit_transform(embedding)

print("original shape:   ", embedding.shape)
print("transformed shape:", embedding_Umap3.shape)

In [ ]:
m= embedding_Umap3 # m is an array of (x,y,z) coordinate triplets

fig = figure()
ax = Axes3D(fig)

aaL = list(aa)
print(len(m))
print(len(aaL))
ax.scatter(m[:,0],m[:,1],m[:,2],color='b') 
for i in range(len(aaL)): #plot each point + it's index as text above
    ax.text(m[i,0],m[i,1],m[i,2],  '%s' % (str(aaL[i])), size=10, zorder=1,  
    color='k') 

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
plt.show()

## Plot the attention layer

I need some serious work on that one 

In [ ]:
model = Model(inputs=model.input,
              outputs=[model.output, model.get_layer('attention_vect').output])
outputs = model.predict(x_valid_pad)
model_outputs = outputs[0]
attention_outputs = outputs[1]

In [ ]:
attention_distrib = np.concatenate( attention_outputs, axis=0 )
plt.hist(attention_distrib, normed=True, bins=50)
plt.ylabel('Attention Distribution')

In [ ]:
# Normalize attention values 
norm_attention = []
for seq in attention_outputs:
    norm_attention.append(seq / np.linalg.norm(seq))

In [ ]:
attention_distrib = np.concatenate( norm_attention, axis=0 )
plt.hist(attention_distrib, normed=True, bins=50)
plt.ylabel('Attention Distribution')

### Check attention for a single protein sequence

In [ ]:
def indices_to_seq(X):
    seq = ''
    for i in range(len(X)):
        seq += int_to_aa[X[i]] 
    return seq

In [ ]:
# I don't fully understand that code
# the color_charvals function doesn't work that well because my vector is big
# and the values are very small, I need a better function

class CharVal(object):
    def __init__(self, char, val):
        self.char = char
        self.val = val

    def __str__(self):
        return self.char

def rgb_to_hex(rgb):
    return '#%02x%02x%02x' % rgb

def color_charvals(s):
    r = 0
    if s.val < 0.07:
        r = 255
    elif s.val > 0.1:
        r = 0
    else:
        r = 255*(1 - (s.val-0.07)/0.05)
    r = int(r)
    color = rgb_to_hex((255, r, 255))
    return 'background-color: %s' % color
    
    #r = int(255/(s.val*10 +1))
    #color = rgb_to_hex((r, r, r))
    #return 'background-color: %s' % color

In [ ]:
X = 6

attention_output = attention_outputs[X][-max_len:]
attention_seq = attention_output[1500-len(x_valid[X]):]
attention = normalize(attention_seq[:,np.newaxis], axis=0).ravel()

# match each aa and attention
char_vals = [CharVal(c, v) for c, v in 
             zip(x_valid[X], attention)]

char_df = pd.DataFrame(char_vals).transpose()
# apply coloring values
char_df = char_df.style.applymap(color_charvals)
char_df

In [ ]:
print(y_valid[X])
print(valid['Lead Protein ID'][X])
print(y_pred_labels[X])

## Select the 'nb_max' maximum attention weights values position through all proteins from the valid dataset and Visualisation

Then, Compare motifs identified to this site: http://elm.eu.org/combined_search?query=NLS

In [ ]:
# Number of highest values to select
nb_max = 50
max_att_indexes = []
for seq in norm_attention:
    max_att_indexes.append(np.sort(np.argpartition(seq, -nb_max)[-nb_max:]))
max_att_nopad = []    
for x in range(len(x_valid)):
    max_att_nopad.append(max_att_indexes[x]-(1500-len(x_valid[x])))

In [ ]:
X = 1

def color_charvals(s):
    r = 255-int(s.val*255)
    color = rgb_to_hex((255,255 , r))
    return 'background-color: %s' % color

max_ind = max_att_nopad[X]
max_vect = np.zeros(len(x_valid[X]))
for val in max_ind:
    max_vect[val] = 1
# match each aa and attention
char_vals = [CharVal(c, v) for c, v in 
             zip(x_valid[X], max_vect)]

char_df = pd.DataFrame(char_vals).transpose()
# apply coloring values
char_df = char_df.style.applymap(color_charvals)
char_df

So... Now I need to look for NLS in valid set and check whether attention catch all of them by comparing sequence indices from max attention

## Get regions deemed important by the attention layer  into the Dataframe

In [ ]:
motifs_loc = []
motifs = []
stop = 0
for l in max_att_nopad:
    start = l[0]
    for i in range(1,len(l)):
        if l[i-1] + 3 >= l[i]:
            stop = l[i]
        else:
            if start < stop:
                motifs.append((start, stop))
            start = l[i]
    if start < stop:
        motifs.append((start, stop))
    motifs_loc.append(motifs)
    motifs = []


In [ ]:
motifs_aa = []
motifs = []
for i in range(len(motifs_loc)):
    seq = x_valid[i]
    m = motifs_loc[i]
    for tup in m:
        motifs.append(seq[tup[0]:tup[1]+1])
    motifs_aa.append(motifs)
    motifs = []

In [ ]:
valid["Model Prediction"] = y_pred_labels

In [ ]:
valid["Motifs"] = motifs_aa
valid["Motifs Localization"] = motifs_loc

In [ ]:
valid['Motifs']

### Description of ELM Regex
http://elm.eu.org/infos/help.html

Regular expression usefull links:
https://www.debuggex.com/cheatsheet/regex/python
https://regex101.com/#python

In [ ]:
NLS_regex = ['[KR][KR].{7,15}[^DE]((K[RK])|(RK))(([^DE][KR])|([KR][^DE]))[^DE]',
             '[^DE]((K[RK])|(RK))[KRP][KR][^DE]',
             '[^DE]((K[RK])|(RK))(([^DE][KR])|([KR][^DE]))(([PKR])|([^DE][DE]))',
             '(([PKR].{0,1}[^DE])|([PKR]))((K[RK])|(RK))(([^DE][KR])|([KR][^DE]))[^DE]']

NES_regex = ['([DEQ].{0,1}[LIM].{2,3}[LIVMF][^P]{2,3}[LMVF].[LMIV].{0,3}[DE])|([DE].{0,1}[LIM].{2,3}[LIVMF][^P]{2,3}[LMVF].[LMIV].{0,3}[DEQ])']

In [ ]:
NLS_valid = []
NLS_nb = 0 # count seq with NLS
NLS_bool = []
for seq in x_valid:
    NLS = []
    for motifs in NLS_regex:
        match = re.finditer(motifs, seq)
        if match:
            for m in match:
                NLS.append((m.start(), m.end()))
    if NLS != []:
        NLS_bool.append(True)
        NLS.sort(key=lambda tup: tup[1])
        NLS_nb +=1
    else:
        NLS_bool.append(False)
    NLS_valid.append(NLS)
                

In [ ]:
print(str(NLS_nb/len(x_valid)*100) + '% of proteins in valid set have at least one NLS motifs')

In [ ]:
valid["NLS"] = NLS_bool

In [ ]:
valid["NLS positions"] = NLS_valid

In [ ]:
def isintersect(a, b):
    for a0 in a:
        for b0 in b:
            if ((b0[0] <= a0[0] and a0[0] < b0[1]) or (a0[0] <= b0[0] and b0[0] < a0[1])):
                return True
    return False
    #
    #a0 = [(i[0], i[1], 0) for i in a]
    #b0 = [(i[0], i[1], 1) for i in b]
    #c = sorted(a0+b0, key=lambda i: i[0])
    #for i in range(len(c)):
    #    j = i+1
    #    while j != len(c) and c[i][1] > c[j][0]:
    #        if (c[i][2] != c[j][2]):
    #            return True
    #        j=j+1
    #return False

assert(isintersect([(0,10)], [(5, 15)])==True)
assert(isintersect([(0,10)], [(10, 20)])==False)
assert(isintersect([(0,10),(20,30)], [(10, 20), (30, 40)])==False)
assert(isintersect([(0,10),(20,30)], [(10, 25), (30, 40)])==True)

In [ ]:
NLS_intersect = []
for i in range(len(motifs_loc)):
    NLS_intersect.append(isintersect(motifs_loc[i],NLS_valid[i]))

In [ ]:
valid["NLS/Attention Intersect"] = NLS_intersect

In [ ]:
valid.groupby(["Global classifier2", "NLS/Attention Intersect"]).count()/valid.groupby("Global classifier2").count()

In [ ]:
isintersect(motifs_loc[0],NLS_valid[0])

In [ ]:
Nuclear = valid.loc[valid['Global classifier2'] == 'Mostly Nuclear']
Nuclear.head()

In [ ]:
valid.head()

In [ ]:
valid.groupby(["Global classifier2", "NLS"]).count()/valid.groupby("Global classifier2").count()

### Same thing with NES

In [ ]:
NES_valid = []
NES_nb = 0 # count seq with NLS
NES_bool = []
for seq in x_valid:
    NES = []
    for motifs in NES_regex:
        match = re.finditer(motifs, seq)
        if match:
            for m in match:
                NES.append((m.start(), m.end()))
    if NES != []:
        NES_bool.append(True)
        NES.sort(key=lambda tup: tup[1])
        NES_nb +=1
    else:
        NES_bool.append(False)
    NES_valid.append(NES)

In [ ]:
print(str(NES_nb/len(x_valid)*100) + '% of proteins in valid set have at least one NES motifs')

In [ ]:
valid["NES"] = NES_bool
valid["NES positions"] = NES_valid
valid.head()

In [ ]:
valid.groupby(["Global classifier2", "NES"]).count()/valid.groupby("Global classifier2").count()

In [ ]:
NES_intersect = []
for i in range(len(motifs_loc)):
    NES_intersect.append(isintersect(motifs_loc[i],NES_valid[i]))

In [ ]:
valid["NES/Attention Intersect"] = NES_intersect

In [ ]:
valid.groupby(["Global classifier2", "NES/Attention Intersect"]).count()/valid.groupby("Global classifier2").count()

# Error Analysis

In [ ]:
# THIS CODE OUTPUT THE INDICES FOR CORRECT AND INCORRECT EXEMPLES.
corrects = []
falses = []
for x in range(len(y_valid)):
    if y_valid_indices[x] == y_pred_indices[x]:
        corrects.append(x)
    else:
        falses.append(x)

In [ ]:
print('There is ' + str(len(corrects)) + ' correctly classified exemples')
print('There is ' + str(len(falses)) + ' incorrectly classified exemples')
print('So the accuracy is: ' + str(len(corrects)/len(x_valid)*100))

In [ ]:
corrects_df = valid[valid.index.isin(corrects)]
falses_df = valid[valid.index.isin(falses)]

In [ ]:
corrects_df.head()

In [ ]:
falses_df.head()

## Checking the model on the test set

In [ ]:
test = pd.read_csv("data/3Classes_test.csv", sep="\t")
y_test = test["Global classifier2"]
y_test_indices = cat_to_indices(y_test)
y_test_OH = convert_to_one_hot(y_test_indices, C = tot_cat)

x_test = test["Sequence"].copy()
x_test_indices = seq_to_indices(x_test, aa_to_int, max_len)
x_test_pad = sequence.pad_sequences(x_test_indices, maxlen=max_len, value=aa_to_int["*"])

In [ ]:
model = load_model('models_attention/1/weights.32-0.54.hdf5')
score = model.evaluate(x_test_pad, y_test_OH)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## Model 2: CNN LSTM with attention 

In [ ]:
max_features = tot_aa + 1
batch_size = 32

#LSTM
units = 128
# Embedding
embedding_size = 48
# Convolution
filters = 20
pool_size = 2

_input = Input(shape=[max_len], dtype='int32')

# get the embedding layer
embedded = Embedding(input_dim=max_features,output_dim=embedding_size,
        input_length=max_len, name = 'embedding_layer', trainable=True, mask_zero=False)(_input)

drop = Dropout(0.2)(embedded)
# Convolution layer of different size

conv2 = Conv1D(filters, 2, padding='same', activation='relu', 
              strides=1)(drop)
drop2 = SpatialDropout1D(0.2)(conv2)
pool2 = MaxPooling1D(pool_size=pool_size)(drop2)

conv3 = Conv1D(filters, 3, padding='same', activation='relu', 
              strides=1)(drop)
drop3 = SpatialDropout1D(0.2)(conv3)
pool3 = MaxPooling1D(pool_size=pool_size)(drop3)

conv5 = Conv1D(filters, 5, padding='same', activation='relu', 
              strides=1)(drop)
drop5 = SpatialDropout1D(0.2)(conv5)
pool5 = MaxPooling1D(pool_size=pool_size)(drop5)

conv9 = Conv1D(filters, 9, padding='same', activation='relu', 
              strides=1)(drop)
drop9 = SpatialDropout1D(0.2)(conv9)
pool9 = MaxPooling1D(pool_size=pool_size)(drop9)

conv12 = Conv1D(filters, 12, padding='same', activation='relu', 
              strides=1)(drop)
drop12 = SpatialDropout1D(0.2)(conv12)
pool12 = MaxPooling1D(pool_size=pool_size)(drop12)

conv16 = Conv1D(filters, 16, padding='same', activation='relu', 
              strides=1)(drop)
drop16 = SpatialDropout1D(0.2)(conv16)
pool16 = MaxPooling1D(pool_size=pool_size)(drop16)

# Concatenate the filters
#concat = concatenate([drop2, drop3, drop5, drop9, drop12], axis=2)
concat = concatenate([pool2, pool3, pool5, pool9, pool12, pool16], axis=2)

#pool = MaxPooling1D(pool_size=pool_size)(concat)

activations = Bidirectional(LSTM(units, return_sequences=True, 
                                 dropout=0.2, recurrent_dropout=0.2))(concat)

# compute importance for each step
attention = Dense(1, activation='tanh')(activations)
attention = Flatten()(attention)
attention = Activation('softmax', name = 'attention_vect')(attention) # attention_vect for visualization
attention = RepeatVector(units*2)(attention)
attention = Permute([2, 1])(attention)
sent_representation = Multiply()([activations, attention])
sent_representation = Lambda(lambda xin: K.sum(xin, axis=-2), output_shape=(units*2,))(sent_representation)

probabilities = Dense(3, activation='softmax')(sent_representation)

model = Model(inputs=_input, outputs=probabilities)

In [ ]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)
d_class_weights = dict(enumerate(class_weights))

model.compile('Nadam', 'categorical_crossentropy', metrics=['accuracy'])
ckpt = ModelCheckpoint(filepath='models_attention/2/CNN_model_weights', 
                           verbose=1, save_best_only=True)
# Train, train, train
history = model.fit(x_train_pad, y_train_OH, 
                    batch_size=32, epochs=60,
                    class_weight = class_weights,
          validation_data=[x_valid_pad, y_valid_OH], callbacks=[ckpt])

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Valid'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Valid'], loc='upper left')
plt.show()

In [ ]:
plot_model(model, to_file='models_attention/2/CNN_model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
model2 = load_model('models_attention/2/CNN_model_weights.hdf5')

In [ ]:
model2.evaluate(x_valid_pad, y_valid_OH, verbose=0)

In [ ]:
y_pred2 = model2.predict(x_valid_pad)
y_pred_indices2 = OH_to_label_indices(y_pred2)
y_pred_labels2 = indices_to_cat(y_pred_indices2)

In [ ]:
conf_mx2 = confusion_matrix(y_valid, y_pred_labels2, labels=cat)

fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(conf_mx2, cmap=plt.cm.gray)
fig.colorbar(cax)
ax.xaxis.set_major_locator(MultipleLocator(1))
ax.yaxis.set_major_locator(MultipleLocator(1))
ax.set_xticklabels([''] + cat,  rotation='vertical')
ax.set_yticklabels([''] + cat)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

### Attention Analysis 

In [ ]:
model2 = Model(inputs=model2.input,
              outputs=[model2.output, model2.get_layer('attention_vect').output])
outputs2 = model2.predict(x_valid_pad)
model_outputs2 = outputs[0]
attention_outputs2 = outputs[1]
attention_outputs2.shape

In [ ]:
# Normalize attention values 
norm_attention2 = []
for seq in attention_outputs2:
    norm_attention2.append(seq / np.linalg.norm(seq))
    
attention_distrib2 = np.concatenate( norm_attention2, axis=0 )
plt.hist(attention_distrib2, normed=True, bins=50)
plt.ylabel('Attention Distribution')

In [ ]:
# So... it has nothing to do with the type of model.. since this is the same I used for 8 compartment analysis

In [ ]:
# pool_size = 2
Attention_1500_2 = []
A = []
for x in norm_attention2:
    for i in range(len(x)):
        A.append(x[i])
        A.append(x[i])
    Attention_1500_2.append(np.array(A))
    A = []
    
# Number of highest values to select
nb_max = 50
max_att_indexes2 = []
for seq in Attention_1500_2:
    max_att_indexes2.append(np.sort(np.argpartition(seq, -nb_max)[-nb_max:]))
max_att_nopad2 = []    
for x in range(len(x_valid)):
    max_att_nopad2.append(max_att_indexes[x]-(1500-len(x_valid[x])))

In [ ]:
X = 1

def color_charvals(s):
    r = 255-int(s.val*255)
    color = rgb_to_hex((255,255 , r))
    return 'background-color: %s' % color

max_ind = max_att_nopad2[X]
max_vect = np.zeros(len(x_valid[X]))
for val in max_ind:
    max_vect[val] = 1
# match each aa and attention
char_vals = [CharVal(c, v) for c, v in 
             zip(x_valid[X], max_vect)]

char_df = pd.DataFrame(char_vals).transpose()
# apply coloring values
char_df = char_df.style.applymap(color_charvals)
char_df

In [ ]:
# Ok so with these model the accuracy is better but it didn't pick up the same regions at all (and for both X=3 is wrong)
# So maybe convolution or pool is not the best idea because I have not clue 

In [ ]:
y_valid[1]

In [ ]:
y_pred_labels[1]

### Test CNN model on test set

In [ ]:
model.evaluate(x_test_pad, y_test_OH)